In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.0 MB/s 
     |████████████████████████████████| 895 kB 52.6 MB/s 
     |████████████████████████████████| 3.3 MB 57.0 MB/s 
     |████████████████████████████████| 59 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 56.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sents = []

with open("/content/drive/MyDrive/event_data_word/test.sent", "r") as f:
  for line in f:
    sents.append(line)

In [ ]:
len(sents)

292

In [ ]:
sentence1=sents[:1]
marked_text="[CLS] " + sentence1 + " [SEP]"
tokenized_text = tokenizer.tokenize(marked_text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

In [ ]:
sentence1=sents[:1]

In [ ]:
sentence1

"British Chancellor of the Exchequer Gordon Brown on Tuesday named the current head of the country 's energy regulator as the new chairman of finance watchdog the Financial Services Authority -LRB- FSA -RRB- .\n"

In [ ]:
marked_text="[CLS] " + sentence1 + " [SEP]"

In [ ]:
tokenized_text = tokenizer.tokenize(marked_text)

In [ ]:
print(tokenized_text)

['[CLS]', 'british', 'chancellor', 'of', 'the', 'exchequer', 'gordon', 'brown', 'on', 'tuesday', 'named', 'the', 'current', 'head', 'of', 'the', 'country', "'", 's', 'energy', 'regulator', 'as', 'the', 'new', 'chairman', 'of', 'finance', 'watch', '##dog', 'the', 'financial', 'services', 'authority', '-', 'l', '##rb', '-', 'f', '##sa', '-', 'rr', '##b', '-', '.', '[SEP]']


In [ ]:
#tokenized_text=list(set(tokenized_text))

In [ ]:
len(tokenized_text) #total words

45

In [ ]:
#text = "After stealing money from the bank vault, the bank robber was seen " \
#       "fishing on the Mississippi river bank."

# Add the special tokens.
#marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
#tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

In [ ]:
len(tokenized_text)

45

In [ ]:
segments_ids = [1] * len(tokenized_text) #to denote which sentence it is a part of.

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [ ]:
segments_tensors = torch.tensor([segments_ids])

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [ ]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i])) #number of sentences
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i])) #number of words
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i])) 

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 45
Number of hidden units: 768


In [ ]:
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())

      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 45, 768])


In [ ]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 45, 768])

In [ ]:
# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 45, 768])

In [ ]:
# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([45, 13, 768])

In [ ]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 45 x 768


In [ ]:
print(len(token_vecs_sum)) #768 dimensioned embedding for each word in the input sentence. (45 words)

45


In [ ]:
len(tokenized_text)

45

In [ ]:
global_embeddings = token_vecs_sum
global_tokens = tokenized_text

In [ ]:
len(sents)

292

In [ ]:
global_embeddings=[]
global_tokens=[]

## Extracting word embeddings from test.sent

In [ ]:
for i in range(0,len(sents)):
  sentence1=sents[i]
  marked_text="[CLS] " + sentence1 + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  # Run the text through BERT, and collect all of the hidden states produced
  # from all 12 layers. 
  with torch.no_grad():
    outputs = model(tokens_tensor)
    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

  token_embeddings = torch.stack(hidden_states, dim=0)
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  token_embeddings = token_embeddings.permute(1,0,2)

  # Stores the token vectors, with shape [22 x 768]
  token_vecs_sum = []

  # `token_embeddings` is a [22 x 12 x 768] tensor.

  # For each token in the sentence...
  for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

  print(i, " loop done")
  global_embeddings.append(token_vecs_sum)
  global_tokens.append(tokenized_text)


0  loop done
1  loop done
2  loop done
3  loop done
4  loop done
5  loop done
6  loop done
7  loop done
8  loop done
9  loop done
10  loop done
11  loop done
12  loop done
13  loop done
14  loop done
15  loop done
16  loop done
17  loop done
18  loop done
19  loop done
20  loop done
21  loop done
22  loop done
23  loop done
24  loop done
25  loop done
26  loop done
27  loop done
28  loop done
29  loop done
30  loop done
31  loop done
32  loop done
33  loop done
34  loop done
35  loop done
36  loop done
37  loop done
38  loop done
39  loop done
40  loop done
41  loop done
42  loop done
43  loop done
44  loop done
45  loop done
46  loop done
47  loop done
48  loop done
49  loop done
50  loop done
51  loop done
52  loop done
53  loop done
54  loop done
55  loop done
56  loop done
57  loop done
58  loop done
59  loop done
60  loop done
61  loop done
62  loop done
63  loop done
64  loop done
65  loop done
66  loop done
67  loop done
68  loop done
69  loop done
70  loop done
71  loop done
72

In [ ]:
print(type(global_embeddings))

<class 'list'>


In [ ]:
print(len(global_embeddings))
print(len(global_embeddings[0]))
print(len(global_embeddings[0][0]))
print(len(global_tokens))
print(len(global_embeddings[0][0]))

292
45
768
292
768


In [ ]:
print(type(global_tokens[0][0]))

<class 'str'>


In [ ]:
# we initialise a global dictionary (storing key value pairs for test, dev and train set) called global dict where key-word, value-tensor of dim 768

global_dict={} #stores only unique values

for i in range(0,len(global_embeddings)):
  for j in range(0,len(global_embeddings[i])):
    key_word=global_tokens[i][j]
    value_tensor=global_embeddings[i][j]
    if key_word in global_dict.keys():
      continue;
    else:
      global_dict.update({key_word:value_tensor})




In [ ]:
print(len(global_dict)) #up till now, test.sent has been stored here.

2611


## Now finding word embeddings from dev.sent

In [ ]:
sents = []

with open("/content/drive/MyDrive/event_data_word/dev.sent", "r") as f:
  for line in f:
    sents.append(line)

In [ ]:
print(len(sents))

340


In [ ]:
#number of words in the first sentence:
print(len(sents[0]))

87


In [ ]:
global_embeddings=[]
global_tokens=[]

In [ ]:
for i in range(0,len(sents)):
  sentence1=sents[i]
  marked_text="[CLS] " + sentence1 + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  # Run the text through BERT, and collect all of the hidden states produced
  # from all 12 layers. 
  with torch.no_grad():
    outputs = model(tokens_tensor)
    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

  token_embeddings = torch.stack(hidden_states, dim=0)
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  token_embeddings = token_embeddings.permute(1,0,2)

  # Stores the token vectors, with shape [22 x 768]
  token_vecs_sum = []

  # `token_embeddings` is a [22 x 12 x 768] tensor.

  # For each token in the sentence...
  for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

  print(i, " loop done")
  global_embeddings.append(token_vecs_sum)
  global_tokens.append(tokenized_text)

In [ ]:
print(len(global_embeddings))
print(len(global_embeddings[0]))
print(len(global_embeddings[0][0]))
print(len(global_tokens))
print(len(global_tokens[0]))
print(type(global_tokens[0][0]))
print(type(global_embeddings[0][0]))

340
23
768
340
23
<class 'str'>
<class 'torch.Tensor'>


In [ ]:
#print((global_tokens[0][4]))
print(len(global_embeddings[0][0]))

768


In [ ]:
# updating global dict

for i in range(0,len(global_embeddings)):
  for j in range(0,len(global_embeddings[i])):
    key_word=global_tokens[i][j]
    value_tensor=global_embeddings[i][j]
    if key_word in global_dict.keys():
      continue;
    else:
      global_dict.update({key_word:value_tensor})




In [ ]:
print(len(global_dict)) #up till now, test.sent and dev.sent has been stored here.

4045


## Extracting word embeddings from train.sent

In [ ]:
sents = []

with open("/content/drive/MyDrive/event_data_word/train.sent", "r") as f:
  for line in f:
    sents.append(line)

In [ ]:
print(len(sents))

3255


In [ ]:
#number of words in the first sentence:
print(len(sents[0]))

243


In [ ]:
global_embeddings=[]
global_tokens=[]

In [ ]:
for i in range(0,len(sents)):
  sentence1=sents[i]
  marked_text="[CLS] " + sentence1 + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  # Run the text through BERT, and collect all of the hidden states produced
  # from all 12 layers. 
  with torch.no_grad():
    outputs = model(tokens_tensor)
    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

  token_embeddings = torch.stack(hidden_states, dim=0)
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  token_embeddings = token_embeddings.permute(1,0,2)

  # Stores the token vectors, with shape [22 x 768]
  token_vecs_sum = []

  # `token_embeddings` is a [22 x 12 x 768] tensor.

  # For each token in the sentence...
  for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

  print(i, " loop done")
  global_embeddings.append(token_vecs_sum)
  global_tokens.append(tokenized_text)

In [ ]:
print(len(global_embeddings))
print(len(global_embeddings[0]))
print(len(global_embeddings[0][0]))
print(len(global_tokens))
print(len(global_tokens[0]))
print(type(global_tokens[0][0]))
print(type(global_embeddings[0][0]))

3255
49
768
3255
49
<class 'str'>
<class 'torch.Tensor'>


In [ ]:
#print((global_tokens[0][4]))
#print(len(global_embeddings[0][0]))

In [ ]:
# updating global dict

for i in range(0,len(global_embeddings)):
  for j in range(0,len(global_embeddings[i])):
    key_word=global_tokens[i][j]
    value_tensor=global_embeddings[i][j]
    if key_word in global_dict.keys():
      continue;
    else:
      global_dict.update({key_word:value_tensor})



In [ ]:
print(len(global_dict)) 

9185


In [ ]:
np_arr = torch_tensor.cpu().detach().numpy()

In [ ]:
tensor_sample=global_embeddings[0][0]

In [ ]:
tensor_sample

tensor([-7.5710e-01, -1.0330e+00, -7.8709e-01,  1.0525e+00, -5.1443e-01,
        -1.3853e+00, -8.2094e-01,  1.4624e-01,  3.5768e-01, -1.0866e+00,
        -1.6580e-01,  1.5161e+00, -8.0159e-01,  2.3410e+00,  1.4964e+00,
         2.1920e+00, -5.4929e-01,  2.3138e-01,  1.2602e+00,  5.9935e-01,
        -9.1138e-01, -8.9620e-01,  2.1713e+00,  1.1568e+00,  1.5384e+00,
        -1.2169e+00, -1.6311e+00, -1.2324e+00, -2.0904e+00,  1.4217e+00,
        -2.7893e-01,  6.2479e-01, -2.9870e-01, -2.0983e+00,  5.1230e-01,
         1.0679e+00, -3.5244e-01, -7.3044e-02,  2.3687e-01, -6.3274e-02,
        -1.3078e+00,  1.3420e+00,  6.2968e-01, -3.8760e-01, -7.3484e-01,
        -4.2292e-01, -1.6711e+01,  1.5709e+00,  2.7951e-01, -3.9117e-01,
         1.5112e+00, -2.1593e+00, -3.6306e-01,  4.0887e-01,  1.2472e+00,
         1.2813e+00, -2.3837e-01, -7.1893e-01, -3.7460e-01, -2.4817e-01,
        -1.1006e-01,  4.0411e-01, -1.6750e+00, -2.1719e-01, -2.1624e+00,
        -4.4270e-01,  1.4765e+00, -5.7474e-01,  5.2

In [ ]:
np_arr = tensor_sample.cpu().detach().numpy()
#print(np_arr)
print(np_arr.shape)

(768,)


In [ ]:
lis_sample = np_arr.tolist()
print(len(lis_sample))
#

768


In [ ]:
print((lis_sample))

[-0.7571009993553162, -1.032955288887024, -0.7870947122573853, 1.0524816513061523, -0.5144277811050415, -1.3852720260620117, -0.8209407329559326, 0.14624309539794922, 0.3576781153678894, -1.0865501165390015, -0.16580255329608917, 1.5160596370697021, -0.8015868067741394, 2.3409523963928223, 1.4964288473129272, 2.192047595977783, -0.5492921471595764, 0.2313826084136963, 1.260237455368042, 0.5993461012840271, -0.9113841652870178, -0.8961969614028931, 2.1712636947631836, 1.1567867994308472, 1.538431167602539, -1.216883897781372, -1.6310796737670898, -1.2323887348175049, -2.0903656482696533, 1.421687126159668, -0.27893170714378357, 0.624790370464325, -0.2987031936645508, -2.0983057022094727, 0.5122988820075989, 1.0679155588150024, -0.3524435758590698, -0.07304379343986511, 0.23687298595905304, -0.06327390670776367, -1.3078484535217285, 1.3419749736785889, 0.629676103591919, -0.38760435581207275, -0.7348389029502869, -0.4229176342487335, -16.710525512695312, 1.5709049701690674, 0.27950999140

In [ ]:
string_sample=" ".join(str(item) for item in lis_sample)

In [ ]:
print(string_sample)

-0.7571009993553162 -1.032955288887024 -0.7870947122573853 1.0524816513061523 -0.5144277811050415 -1.3852720260620117 -0.8209407329559326 0.14624309539794922 0.3576781153678894 -1.0865501165390015 -0.16580255329608917 1.5160596370697021 -0.8015868067741394 2.3409523963928223 1.4964288473129272 2.192047595977783 -0.5492921471595764 0.2313826084136963 1.260237455368042 0.5993461012840271 -0.9113841652870178 -0.8961969614028931 2.1712636947631836 1.1567867994308472 1.538431167602539 -1.216883897781372 -1.6310796737670898 -1.2323887348175049 -2.0903656482696533 1.421687126159668 -0.27893170714378357 0.624790370464325 -0.2987031936645508 -2.0983057022094727 0.5122988820075989 1.0679155588150024 -0.3524435758590698 -0.07304379343986511 0.23687298595905304 -0.06327390670776367 -1.3078484535217285 1.3419749736785889 0.629676103591919 -0.38760435581207275 -0.7348389029502869 -0.4229176342487335 -16.710525512695312 1.5709049701690674 0.2795099914073944 -0.39116790890693665 1.511204719543457 -2.1

In [ ]:
full_string=global_tokens[0][4] + " "+string_sample
print(full_string)

secretary -0.7571009993553162 -1.032955288887024 -0.7870947122573853 1.0524816513061523 -0.5144277811050415 -1.3852720260620117 -0.8209407329559326 0.14624309539794922 0.3576781153678894 -1.0865501165390015 -0.16580255329608917 1.5160596370697021 -0.8015868067741394 2.3409523963928223 1.4964288473129272 2.192047595977783 -0.5492921471595764 0.2313826084136963 1.260237455368042 0.5993461012840271 -0.9113841652870178 -0.8961969614028931 2.1712636947631836 1.1567867994308472 1.538431167602539 -1.216883897781372 -1.6310796737670898 -1.2323887348175049 -2.0903656482696533 1.421687126159668 -0.27893170714378357 0.624790370464325 -0.2987031936645508 -2.0983057022094727 0.5122988820075989 1.0679155588150024 -0.3524435758590698 -0.07304379343986511 0.23687298595905304 -0.06327390670776367 -1.3078484535217285 1.3419749736785889 0.629676103591919 -0.38760435581207275 -0.7348389029502869 -0.4229176342487335 -16.710525512695312 1.5709049701690674 0.2795099914073944 -0.39116790890693665 1.5112047195

## Printing the values into a text file
Format: First line has [number of words][space][dimension of each embedding],
and each line has [word] followed by the embedding vector values. 

In [ ]:
with open('/content/drive/MyDrive/Bert_embeddings.txt', 'w') as f:
    f.write(str(len(global_dict))+" 768")
    f.write('\n')
    for key, value in global_dict.items():
      np_arr = value.cpu().detach().numpy()
      lis = np_arr.tolist()
      string_embedding=" ".join(str(item) for item in lis)
      full_string=key + " " + string_embedding
      f.write(full_string)
      f.write('\n')